# Exercise Notebook on Interacting with Databases

In [1]:
import pandas as pd
import numpy as np
import os.path

In this exercise notebook we will work with a SQLite database. First, let's create a new database from the *movielens* dataset. The ratings data are very large, ingesting those into SQLite is quite slow. Let's only take the first 100 thousand records.

In [3]:
# DO NOT MODIFY

# set here the relative path to the movielens folder
MOVIELENS="../../Week-4-Pandas/movielens"

movies = pd.read_csv(os.path.join(MOVIELENS, 'movies.csv'), sep=',')
ratings = pd.read_csv(os.path.join(MOVIELENS, 'ratings.csv'), sep=',')

In [4]:
# DO NOT MODIFY

ratings = ratings.iloc[:100000]

We first create on disk the SQLite database `movielens.db` and then we use the `to_sql` method of DataFrames to store the data there, we specify that is the table exists already, it should be replaced.

In [5]:
# DO NOT MODIFY

import sqlite3
conn = sqlite3.connect("movielens.db")

In [6]:
print("Size of the database is {:.2g} MB".format(os.path.getsize("movielens.db")/1024**2))

Size of the database is 0 MB


In [10]:
# DO NOT MODIFY

movies.to_sql("movies", conn, if_exists="replace")

In [11]:
# DO NOT MODIFY

ratings.to_sql("ratings", conn, if_exists="replace")

In [12]:
print("Size of the database is {:.2g} MB".format(os.path.getsize("movielens.db")/1024**2))

Size of the database is 7.8 MB


## Exercise 1: get the names of all the tables in the database 

First let's find out the name of the tables available inside the database

In [13]:
cursor = conn.cursor()

In [21]:
def get_list_of_tables(sqlite_cursor):
    """Return a list of table names in a sqlite database
    
    Parameters
    ----------
    sqlite_cursor : sqlite3.Cursor
    SQLite database cursor
    
    Returns
    -------
    tables : list of strings
    list of table names"""
### BEGIN SOLUTION
    tables = []
    for row in sqlite_cursor.execute("SELECT name FROM sqlite_master"):
        tables.append(row[0])
    return tables
### END SOLUTION

In [22]:
movielens_db_tables = get_list_of_tables(cursor)
movielens_db_tables

['movies', 'ix_movies_index', 'ratings', 'ix_ratings_index']

In [23]:
# DO NOT MODIFY

assert isinstance(movielens_db_tables[0], str), "Make sure that the function does not return a list of tuples but just a list of strings"
assert "movies" in movielens_db_tables, "It looks like the movies table is missing"
assert "ratings" in movielens_db_tables, "It looks like the ratings table is missing"

## Exercise 2: Print a few ratings records

Let's use the `sqlite3` package to extract the first 3 ratings from the ratings table.

In [27]:
first_3_ratings = None
### BEGIN SOLUTION
first_3_ratings = cursor.execute("SELECT * FROM sqlite_master LIMIT 3").fetchall()
### END SOLUTION

In [28]:
first_3_ratings

[('table',
  'movies',
  'movies',
  2,
  'CREATE TABLE "movies" (\n"index" INTEGER,\n  "movieId" INTEGER,\n  "title" TEXT,\n  "genres" TEXT\n)'),
 ('index',
  'ix_movies_index',
  'movies',
  584,
  'CREATE INDEX "ix_movies_index"ON "movies" ("index")'),
 ('table',
  'ratings',
  'ratings',
  1043,
  'CREATE TABLE "ratings" (\n"index" INTEGER,\n  "userId" INTEGER,\n  "movieId" INTEGER,\n  "rating" REAL,\n  "timestamp" INTEGER\n)')]

In [29]:
assert len(first_3_ratings) == 3, "Make sure you are selecting only 3 rows"
assert len(first_3_ratings[0]) == 5, "Something went wrong, each row of ratings should have 5 columns"

Explore more the first_3_ratings data structure, what kind of Python object is it? Find out with `type()`.
What kind of Python object is each element of it?

## Exercise 3: Load the ratings table in Pandas

Let's load the entire ratings table into a dataframe

In [32]:
ratings_dataframe = None
### BEGIN SOLUTION
ratings_dataframe = pd.read_sql_query("SELECT * FROM ratings", conn)
### END SOLUTION

In [33]:
assert len(ratings_dataframe) == 100000, "Check your code, you should have 100000 rows"

## Exercise 4: Load a subset of the movies table in Pandas

We would like now to only load the movies of the Fantasy genre into a Pandas dataframe, not the complete movies table. This is very useful for large datasets because we can save a lot of memory if you avoid doing the filtering in pandas but let the database take care of it.

Let's first introduce the SQL syntax for checking if a records contains a string, for example let's print the first 3 Children movies:

In [45]:
cursor.execute("SELECT * FROM movies WHERE genres LIKE '%Children%' LIMIT 3").fetchall()

[(0, 1, 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'),
 (1, 2, 'Jumanji (1995)', 'Adventure|Children|Fantasy'),
 (7, 8, 'Tom and Huck (1995)', 'Adventure|Children')]

In [66]:
fantasy_movies_dataframe = None
### BEGIN SOLUTION
fantasy_movies_dataframe = pd.read_sql_query("SELECT * FROM movies WHERE genres LIKE '%Fantasy%'", conn)
### END SOLUTION
fantasy_movies_dataframe

,index,movieId,title,genres
0,0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,2,Jumanji (1995),Adventure|Children|Fantasy
2,28,29,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
3,43,44,Mortal Kombat (1995),Action|Adventure|Fantasy
4,55,56,Kids of the Round Table (1995),Adventure|Children|Comedy|Fantasy
...,...,...,...,...
2726,62299,208679,Deer Boy (2017),Drama|Fantasy
2727,62301,208683,The Man Without Gravity (2019),Fantasy
2728,62365,208909,Super Xuxa Contra o Baixo Astral (1988),Children|Comedy|Fantasy
2729,62385,209041,Yamasong: March of the Hollows (2017),Animation|Fantasy


In [67]:
# DO NOT MODIFY

assert len(fantasy_movies_dataframe) == 1412, "Make sure you do not have LIMIT in the SQL query"
assert fantasy_movies_dataframe.genres.str.contains("Fantasy").all(), "All rows should contain the Fantasy string, try to identify which rows are not"

AssertionError: Make sure you do not have LIMIT in the SQL query

In the previous cell, we are using the `.all()` DataFrame method, what is it doing? look for its documentation and play with it!